# How to check available remote hosts and if they are properly configured

* **Difficulty level**: difficult
* **Time need to lean**: 30 minutes or more
* **Key points**:
 
  

## Host configuration

### Editing host configuration files

The configuration files are in YAML format. You can edit them manually, or use the `sos config` command to add or modify settings. For example, command

```bash
sos config --hosts --set hosts.shark.address username@shark.com
```

would write to `~/.sos/hosts.yml` the following content

```
hosts:
    shark:
        address: username@shark.com
```

This effectively defines a host with alias `shark` and an `address` with value `username@shark.com`.

As described in section [`SoS Syntax`](SoS_Syntax.html), SoS provides a few features to help maintain and use of host configurations.

First, dictionary `CONFIG` is merged from several configuration files. This allows you to use, for example, a general host configuration file for all hosts, and make local modifications in host-specific `~/.sos/config.yml` even project-specific configuration file.

Second, SoS allows you to use string interpolation for keys defined in `CONFIG`. For example, you can define arbitrary key value pairs for the interpolation of commands and templates. For example, if you could define `queue` in `hosts.cluster` as follows

```
hosts:
   cluster:
      queue: long
      job_template: |
#PBS -q {queue}
...
```

and `queue` will be used in `job_template`.

If the name of a variable matches a runtime variable (e.g. `mem`, `walltime`), its value would be overriden by corresponding runtime variable. This feature can be used to define default values for runtime variables. For example,

```
hosts:
    cluster:
        walltime: '10:00:00'
        job_template: |
    #PBS -l walltime={walltime}
```

would set a default `walltime` of 10 hours for tasks without specified `walltime`.

Formally speaking, strings in SoS configuration file, if interpolated, would be interpolated with a local namespace that is the dictionary in which the key exists, and a global namespace that is the complete `CONFIG` dictionary. That is to say, if you have a configuration file
  ```
  user_name: bpeng
  hosts:
    cluster:
      address: "{user_name}domain.com:{port}"
      port: 123
  ```
`hosts['cluster']['address']` would be interpolated with `port` from the `hosts['cluster']` and `user_name` from the top level configuration. 

To make it easy to share the `hosts.yml` file among users in the same group, SoS automatically defines `user_name` to local user ID if it is not explicitly defined so a typical `hosts.yml` would contain definitions such as
  ```
  hosts:
    cluster:
      address: "{user_name}@domain.com"
      paths:
        home: "/home/{user_name}"
    desktop:
      paths:
        home: c:\Users\{user_name}
     
  ```
with `{user_name}` automatically determined for each user.

Finally, a special `based_on` key could be used to create configuration based on one or more other dictionaries. For example, you could define two queues `short` and `long` with `long` based on definitions of `short`.

```
hosts:
  short:
    queue: short
    job_template: |
        {queue}
  long:
    based_on: hosts.short
    queue: long
```

<div class="alert alert-warning">
  <strong>Hint:</strong> Command <br>
  <pre>% sos config --global --set hosts.short_name.based_on hosts.very_long_name
</pre>
can add
<pre>
hosts:
    short_name:
       based_on: hosts.very_long_name
</pre>
to <pre>~/.sos/config.yml</pre>, which effectively creates an alias <code>short_name</code>
for <code>very_long_name</code>.
</div>

### Common host configuration

SoS needs to know how to connect to a remote host, how to synchronize files between local and remote hosts, and how to execute commands on the remote host. These should be defined with the following keys:

#### `address`

IP address or URL of the host. Note that

* `address` should be ignored for hosts, for example your desktop, that do not accept remote execution.
* If you have a different user name on the the remote host, specify the `address` in the format of `username@hostaddress`.
* SoS does not support username/password authentication so **public key authentication between local and remote hosts is reuired for communication between local and remote host**.
* SoS currently does not support remote execution on windows hosts so no `address` is needed for windows hosts.

#### `hostname`

The `hostname` of the machine, as reported by command `hostname` or Python's `socket.gethostname()`. This entry is used to identify the machine on which sos starts. It is needed only if the hostname is different from `alias` and `address`.

#### `port`

ssh port of the remote host, which is `22` by default. 

#### `shared`

Option `shared` tells SoS which file systems are shared between localhost and some remote hosts so that it does not have to synchronize files under these directories between the hosts.

The `shared` entry should be defined with `name` and `path` pairs in the format of

```
hosts:
    desktop:
    server:
        shared:
            project: /myprojects
            HTFS: /
    worker:
        shared:
            HTFS: /
    server1:
        shared:
            project: /scratch/myprojects
            data: /scratch/data
````

The above cooked configuration says:

1. `desktop` does not share any volume with any other machine so all files need to be transferred.
2. `server` and `worker` shares `HTFS` with directory `/`, so all files are shared.
3. `server` and `server1` share a `project` volume but the volume is mounted at different locations. So files under `myprojects` are not synchronized if you are submitting jobs from `server` to `server1`, and files under `/scratch/myprojects` are not synchronized if you are submitting jobs from `server1` to `server`.


#### `paths`

`paths` defines paths that will be translated when a task is executed remotely. For example, your input file on a mac might be `/Users/myuser/project/KS28.fa`, but it should be named `/home/myuser/project/KS28.fa` if it is processed on a remote server. In this case, you should define directories `/Users/myuser` and `/home/myuser` as equivalent directories on the two hosts, using 

```
hosts:
    desktop:
        paths:
            home: /Users/myuser
    server:
        paths:
            home: /home/myuser
```

Multiple entried could be defined and the files would be mapped by the longest mapping path. For example, if you have on the server a shared location for all resources, you could define

```
hosts:
    desktop:
        paths:            
            home: /Users/myuser
            resource: /Users/myuser/resources
    server:
        paths:
            home: /home/myuser
            resource: /shared/resources
```

so that `/Users/myuser/resources/hg19.fa` could be mapped to `/shared/resources/hg19.fa` on the server. Note that `/Users/myuser/resource/hg19.fa` would be mapped to `resource` instead of `home` because `resource` matches longer piece of the input path.

A remote host can be accessible from a local host only if the remote host defines all paths defined by the local host. More specifically, if host A defines path `home` and host B defines paths `home` and `resource`, it is possible to connect from host A to host B using `home`, but not from host B to A because SoS does not know how to map paths under `resource`.

#### `path_map` (derived)

With definitions of `shared` and `paths` on both hosts, SoS would derive a set of `path_maps` between all hosts using common keys in `shared` and `path` between the hosts. Actually, when you run 

```
sos status -q -v3
```

to list all host configurations, SoS would list all hosts accessible from `localhost`, with host-specific `path_map`, which is a list of directory mappings between local and remote directories. For example, the `path_map` from `desktop` to `server` using the above example would be

```
/Users/myuser -> /home/myuser
/Users/myuser/resoruces -> /shared/resources
```

Note that if a directory shows in both `shared` and `paths` (e.g. `/scratch` in `shared` and `/scratch/user` in `paths`), files can still be synchronized following `path_map` to a different directory even if they are shared and are already on the remote server.

### Host preparation

To execute a sos workflow or task on a remote host, you will need to prepare the host as follows:

1. Set up public key authentication between local and remote host so that you can log in to remote host without being prompted for password.
2. Install SoS on both local and remote hosts.
3. Make sure you have `ssh`, `scp`, and `rsync` installed on local host. For windows, you might want to install these tools with [msys2](http://www.msys2.org/) or enable Ubuntu Linux Shell (windows 10 only, need to turn on developer mode).
4. Set up `$PATH` on the remote host to make sure sos (and other commands that you would like to execute) could be executed remotely with commands like
```
% ssh remote-host "bash --login -c 'sos -h'"
```
sos uses `bash --login` to execute commands on remote host to make sure `$PATH` etc are properly set up. This requires `bash` command on remote host so it is not yet possible to execute workflows on a windows host.


The first step is pretty routine but can be tedious if you have multiple hosts. sos provides a small utility

```
sos remote setup address_or_alias
```
to create a public key (if needed), copy the key to remote host, and append the key to `.ssh/authorized_keys` on the remote host. The command will prompt for a password or use a  password provided with command line option `--password`. This command can set up more hosts, or all hosts defined in SoS configuration files if no host is specified.

### Host configurations

SoS learns how to interact with the remote hosts from its configuration files. If you are using a JupyterHub server, your system admin might have already set up some hosts for you in a site configuration file. Otherwise you will need to add host configurations to your own host definition file `~/.sos/hosts.yml`. Note that this file is the preferred configuration file for host definitions but you can also define them in `~/.sos/config.yml`, or any configuration file that you can specify with command line option `-c` (`--config`).

To check what hosts have been defined for you, you can use command

```
% sos remote list
Alias  Addre           Queue Type Description
-----  -----           ---------- -----------
bcb    name.remote.edu process
laptop localhost       process
```

to list all defined hosts, or

```
% sos config
{ 'hosts': { 'bcb': { 'address': 'name.remote.edu',
                      'paths': {'home': '/Users/bpeng1'},
             'laptop': { 'address': 'localhost',
                         'paths': {'home': '/Users/bpeng1'}}},                    
  'localhost': 'laptop',
  'user_name': 'bpeng1'}
```
to check the content of your SoS configurations. As you can see, the hosts `bcb` and `laptop` are defined under a key `hosts` in the configuration files.

SoS needs to know both the client (your local machine) and server to be able to send local jobs to remote servers or task queues. SoS determines the machine you are on through

1. Check `hostname` of the machine against `hostname`, `alias`, or the `address` of the host.
2. Check the IP address against `address` if an IP address is used for the `address`.
3. Check the existence of a `localhost` setting, which should be one of the aliases of the `hosts`. Because my laptop does not have a fixed hostname, I defined `localhost` as `laptop` to let SoS know the machine I am working now.

Before you submit any job to a remote host, it is highly recommended that you run

```
% sos remote test alias
```
to test if the remote host (`alias` as keys under `hosts` in SoS configuration) is usable. If you see any `FAIL` from the output, you can run
```
% sos remote test alias -v3
```
to get more details on the error and command
```
% sos remote setup alias
```
can help setup public-key access to remote host.

<div class="alert alert-warning">
  <strong>Hint:</strong> Command <br>
  <pre>% sos remote test
</pre>
can check all defined hosts.
</div>

## System setup

### Public-key access

Following any online tutorial, set up public-key access from your local machine to the remote host. If your public key does not work, check file permissions of `~/.ssh`, files under `.ssh`, and `$HOME` in some cases. After setting up the server, make sure you can login without password using command

```bash
% ssh remote-host
```

### Software installation

You will need to install the latest version of sos (preferrably identical version between local and remote hosts), and the software you will need to run. Test it by logging to the remote machine with commands

```bash
% sos -h
```

### Check `$PATH`

Commands that are available in login shell are not necessarily available during remote execution. Basically, remote execution through `ssh` invokes a non-interactive and non-login shell with basic `$PATH`. SoS tries to address this problem by executing commands through a login shell

```bash
% ssh remote-host "bash --login -c 'sos execute task_id'"
```

However, default `.bashrc` on the remote server might contain a line like

```bash
[ -z "$PS1" ] && return
```

that makes it exit when `bash` is not running interactively. This line has to be removed in order to have complete `$PATH` during remote execution.

Now, fire command

```bash
% ssh remote-host "bash --login -c 'sos -h'"
```

from your local machine and see if `sos` can be invoked. Similarly, verify if the command you would like to execute remotely can be executed in this manner.

## Further reading

* 